In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://tv360.vn/collections/movies/phim-au-my?c=1135"

# Headers to simulate a real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def get_movie_links():
    """Fetch all movie links from the main movie page."""
    response = requests.get(BASE_URL, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    movie_links = []
    movie_containers = soup.find_all("div", class_="css-16dzf3w")  # Locate movie container
    print(len(movie_containers))
    
    for container in movie_containers:
        links = container.find_all("a", href=True)  # Find all <a> tags with href
        for link in links:
            full_url = "https://tv360.vn" + link["href"]
            movie_links.append(full_url)

    return movie_links



movie_links = get_movie_links()
print(f"Found {len(movie_links)} movies. Crawling details...")

24
Found 24 movies. Crawling details...


In [8]:
movie_links[0]

'https://tv360.vn/movie/4k-lang-quen-oblivion?m=24605&col=1135&sect=COLLECTION&page=list_col'

In [ ]:
import re

import requests
from bs4 import BeautifulSoup

BASE_URL = ["https://tv360.vn/collections/movies/phim-au-my?c=1135", "https://tv360.vn/collections/movies/phim-hai?c=1123", "https://tv360.vn/collections/movies/phim-hoat-hinh?c=1129", "https://tv360.vn/collections/movies/anime?c=1126", "https://tv360.vn/collections/movies/phim-chau-a?c=1132", "https://tv360.vn/collections/movies/phim-tam-ly?c=1144", "https://tv360.vn/collections/movies/phim-trung-quoc?c=1153"]

# Headers to simulate a real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def get_movie_links():
    movie_links = []
    for link in BASE_URL:
        """Fetch all movie links from the main movie page."""
        response = requests.get(link, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        
        movie_containers = soup.find_all("div", class_="css-16dzf3w")  # Locate movie container
        print(len(movie_containers))
        
        for container in movie_containers:
            links = container.find_all("a", href=True)  # Find all <a> tags with href
            for link in links:
                full_url = "https://tv360.vn" + link["href"]
                movie_links.append(full_url)
    return movie_links



movie_links = get_movie_links()
print(f"Crawled {len(movie_links)} links")

def crawl_movie_page(url):
    """Visit a movie's page and extract information."""
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract movie details from div with class "css-1hvy2us"
    movie_info_div = soup.find("div", class_="css-1hvy2us")
    if movie_info_div:
        return movie_info_div.get_text(separator="\n", strip=True)
    return "No information found"

def extract_movie_info(data):
    info = {}
    
    # Title
    info['title'] = data[0]
    
    # Category (Từ phần tử thứ 2 đến khi gặp một phần tử bắt đầu bằng title)
    title_lower = info['title'].lower()
    category_end_idx = next((i for i, s in enumerate(data[1:], start=1) if s.lower().startswith(title_lower)), len(data))
    info['category'] = data[1:category_end_idx]
    
    # Plot (Từ phần tử có title đến khi gặp "Diễn viên:")
    plot_start_idx = category_end_idx
    plot_end_idx = next((i for i, s in enumerate(data[plot_start_idx:], start=plot_start_idx) if "Diễn viên:" in s), len(data))
    info['plot'] = " ".join(data[plot_start_idx:plot_end_idx]).strip()
    
    # Extract directors, actors, country
    def extract_section(keyword, end_keywords):
        try:
            start_idx = data.index(keyword) + 1
            end_idx = next((i for i, s in enumerate(data[start_idx:], start=start_idx) if s in end_keywords), len(data))
            return [s.strip(',') for s in data[start_idx:end_idx]]
        except ValueError:
            return []
    
    info['actors'] = extract_section("Diễn viên:", ["Đạo diễn:", "Quốc gia:"])
    info['director'] = extract_section("Đạo diễn:", ["Quốc gia:"])
    info['country'] = extract_section("Quốc gia:", [])
    
    return info



24
24
24
24
24
24
24
Crawled 168 links


In [23]:
data = []
for movie_url in movie_links:
    movie_info = crawl_movie_page(movie_url)
    movie_info = movie_info.split("\n")
    exclude_list = [
    "T18", "Xem sau", "Quản lý thiết bị", "Đóng", "Đăng xuất", 
    "Chia sẻ", "Nhúng", "Facebook", "Whatsapp", "Telegram", 
    "Viber", "Email", "Sao chép", "Bắt đầu tại", "Mới trên TV360"
    ]
    movie_info = [item for item in movie_info if item not in exclude_list]
    details = extract_movie_info(movie_info)
    data.append(details)

In [24]:
len(data)

168

In [25]:
import pandas as pd

df = pd.DataFrame(data)

# Display the first few rows
print(df.head())

# Optionally, save it to a CSV file
df.to_csv("movies_data.csv", index=False)

                                        title  \
0                     4K Lãng Quên - Oblivion   
1             Đêm Ở Trường Học - Night School   
2                    Hành Trình Trở Về - Home   
3                    Siêu Anh Hùng - Superwho   
4  4K Sự Nổi Dậy Hoàn Hảo 3 - Pitch Perfect 3   

                                            category  \
0  [T16, Phim Âu Mỹ, Hành động & Phiêu lưu, Lãng ...   
1                     [T13, Phim Âu Mỹ, Phim tâm lý]   
2                    [P, Phim Âu Mỹ, Phim hoạt hình]   
3  [T13, Phim Âu Mỹ, Hành động & Phiêu lưu, Diễn ...   
4  [T13, Phim Âu Mỹ, Romance, Sự Nổi Dậy Hoàn Hảo...   

                                                plot  \
0                                                      
1  Đêm Ở Trường Học - Night School là một bộ phim...   
2  Hành Trình Trở Về - Home là một bộ phim hoạt h...   
3                                                      
4                                                      

                               

In [18]:
import re

def extract_movie_info(data):
    info = {}
    
    # Title
    info['title'] = data[0]
    
    # Category (Từ phần tử thứ 2 đến khi gặp một phần tử bắt đầu bằng title)
    title_lower = info['title'].lower()
    category_end_idx = next((i for i, s in enumerate(data[1:], start=1) if s.lower().startswith(title_lower)), len(data))
    info['category'] = data[1:category_end_idx]
    
    # Plot (Từ phần tử có title đến khi gặp "Diễn viên:")
    plot_start_idx = category_end_idx
    plot_end_idx = next((i for i, s in enumerate(data[plot_start_idx:], start=plot_start_idx) if "Diễn viên:" in s), len(data))
    info['plot'] = " ".join(data[plot_start_idx:plot_end_idx]).strip()
    
    # Extract directors, actors, country
    def extract_section(keyword, end_keywords):
        try:
            start_idx = data.index(keyword) + 1
            end_idx = next((i for i, s in enumerate(data[start_idx:], start=start_idx) if s in end_keywords), len(data))
            return [s.strip(',') for s in data[start_idx:end_idx]]
        except ValueError:
            return []
    
    info['actors'] = extract_section("Diễn viên:", ["Đạo diễn:", "Quốc gia:"])
    info['director'] = extract_section("Đạo diễn:", ["Quốc gia:"])
    info['country'] = extract_section("Quốc gia:", [])
    
    return info

# Example data sets
movies_data = [
    ['4K Lãng Quên - Oblivion', 'T16', 'Phim Âu Mỹ', 'Hành động & Phiêu lưu', 'Lãng Quên - Oblivion: Một người lính dày dặn được giao nhiệm vụ khai thác những tài nguyên còn lại của Trái đất bắt đầu hoài nghi về nhiệm vụ và sự tồn tại của mình.', 'Diễn viên:', 'Morgan Freeman', ',', 'Olga Kurylenko', 'Đạo diễn:', 'Joseph Kosinski', 'Quốc gia:', 'United States of America'],
    ['Đêm Ở Trường Học - Night School', 'T13', 'Phim Âu Mỹ', 'Phim tâm lý', 'Đêm Ở Trường Học - Night School là một bộ phim hài kể về Teddy Walker, một người đàn ông thất nghiệp phải quay lại trường học để lấy bằng tốt nghiệp trung học sau khi mất việc. Anh tham gia một lớp học ban đêm với hy vọng cải thiện cuộc sống của mình, nhưng gặp phải nhiều tình huống dở khóc dở cười cùng với nhóm bạn học đầy cá tính và cô giáo cứng rắn Carrie.', 'Diễn viên:', 'Kevin Hart', ',', 'Taran Killam', ',', 'Rob Riggle', ',', 'Romany Malco', ',', 'Tiffany Haddish', 'Đạo diễn:', 'Malcolm D. Lee', 'Quốc gia:', 'United States of America'],
    ['Hành Trình Trở Về - Home', 'P', 'Phim Âu Mỹ', 'Phim hoạt hình', 'Hành Trình Trở Về - Home là một bộ phim hoạt hình kể về cuộc gặp gỡ giữa một cô bé tên Tip và một người ngoài hành tinh ngộ nghĩnh tên Oh, thuộc chủng loài Boov đang tìm cách chiếm đóng Trái Đất.', 'Diễn viên:', 'Jennifer Lopez', ',', 'Steve Martin', ',', 'Brian Stepanek', ',', 'Rihanna', ',', 'Jim Parsons.', ',', 'Matt Jones', 'Đạo diễn:', 'Tim Johnson', 'Quốc gia:', 'United States of America']
]

# Process each movie
details = [extract_movie_info(movie) for movie in movies_data]

# Print results
for movie in details:
    print(movie)


{'title': '4K Lãng Quên - Oblivion', 'category': ['T16', 'Phim Âu Mỹ', 'Hành động & Phiêu lưu', 'Lãng Quên - Oblivion: Một người lính dày dặn được giao nhiệm vụ khai thác những tài nguyên còn lại của Trái đất bắt đầu hoài nghi về nhiệm vụ và sự tồn tại của mình.', 'Diễn viên:', 'Morgan Freeman', ',', 'Olga Kurylenko', 'Đạo diễn:', 'Joseph Kosinski', 'Quốc gia:', 'United States of America'], 'plot': '', 'actors': ['Morgan Freeman', '', 'Olga Kurylenko'], 'director': ['Joseph Kosinski'], 'country': ['United States of America']}
{'title': 'Đêm Ở Trường Học - Night School', 'category': ['T13', 'Phim Âu Mỹ', 'Phim tâm lý'], 'plot': 'Đêm Ở Trường Học - Night School là một bộ phim hài kể về Teddy Walker, một người đàn ông thất nghiệp phải quay lại trường học để lấy bằng tốt nghiệp trung học sau khi mất việc. Anh tham gia một lớp học ban đêm với hy vọng cải thiện cuộc sống của mình, nhưng gặp phải nhiều tình huống dở khóc dở cười cùng với nhóm bạn học đầy cá tính và cô giáo cứng rắn Carrie.', 